<a href="https://colab.research.google.com/github/MtHenriqueF/Customer-segmentation-clustering/blob/Pre-processamento/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução

Usaremos os arquivos CSV disponíveis em: [customer-segmentation](https://www.kaggle.com/datasets/kaushiksuresh147/customer-segmentation)

Esse dataset foi criado para classificação entre grupos A, B, C e D, porém utilizarei ele para aplicar modelo de clustering, segue uma breve descrição:


#### **Context**
An automobile company has plans to enter new markets with their existing products (P1, P2, P3, P4, and P5). After intensive market research, they’ve deduced that the behavior of the new market is similar to their existing market.

In their existing market, the sales team has classified all customers into 4 segments (A, B, C, D ). Then, they performed segmented outreach and communication for a different segment of customers. This strategy has work e exceptionally well for them. They plan to use the same strategy for the new markets and have identified 2627 new potential customers.

You are required to help the manager to predict the right group of the new customers.



# Carregando e configurando dataset

## Carregando

In [3]:
import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("kaushiksuresh147/customer-segmentation")

print("Path to dataset files:", path)

print("Arquivos disponíveis:")
print(os.listdir(path))


100%|██████████| 99.9k/99.9k [00:00<00:00, 51.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/kaushiksuresh147/customer-segmentation/versions/31
Arquivos disponíveis:
['Train.csv', 'Test.csv']


## Combinando treino e teste

In [4]:
import pandas as pd
import os


train_path = os.path.join(path, "Train.csv")
test_path = os.path.join(path, "Test.csv")

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)



In [5]:
print("--- Informações Iniciais ---")
print(f"Shape do df_train: {df_train.shape}")
print(f"Shape do df_test: {df_test.shape}")
print("\nColunas em Train.csv:")
print(df_train.columns)
print("\nColunas em Test.csv:")
print(df_test.columns)


--- Informações Iniciais ---
Shape do df_train: (8068, 11)
Shape do df_test: (2627, 11)

Colunas em Train.csv:
Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')

Colunas em Test.csv:
Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')


In [6]:
df_combined = pd.concat([df_train, df_test], ignore_index=True)

print(f"Shape do df_combined: {df_combined.shape}")
print(f"Total de linhas esperado: {df_train.shape[0] + df_test.shape[0]}")


Shape do df_combined: (10695, 11)
Total de linhas esperado: 10695


## Removendo ID

ID é um péssimo atributo para qualquer modelo não supervisionado, pois não características de agrupamento que podem ser tiradas de um atributo que deve ser único para qualquer linha

In [7]:
df = df_combined.drop("ID", axis=1)

In [8]:
df

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...
10690,Male,No,29,No,Healthcare,9.0,Low,4.0,Cat_6,B
10691,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,A
10692,Female,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6,C
10693,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4,C


## Removendo Target

Como o dataset original já era clusterizado, iremos tirar o agrupamento feito

In [9]:
df = df.drop("Segmentation", axis=1)

# Baseline

Antes de qualquer análise vou criar um caso base onde utilizarei os métodos mais simples possíveis para clusterização.

Usarei moda, normalização dos dados e kmeans apenas para efeitos de comparação com modelos mais complexos de imputação, remoção de outliers e método de agrupamento

In [13]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_features = ['Age', 'Work_Experience', 'Family_Size']
ordinal_features = ['Spending_Score']
nominal_features = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Var_1']

#Definir a ordem para a variável ordinal
spending_order = ['Low', 'Average', 'High']


#Pipeline para dados numéricos: imputa com a mediana e depois normaliza
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

#Pipeline para dados ordinais: imputa com a moda e depois codifica
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=[spending_order]))
])

#Pipeline para dados nominais: imputa com a moda e depois aplica One-Hot Encoding
#OHE pode ser desnecessario caso use kprototypes
nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('nom', nominal_transformer, nominal_features)
    ],
    remainder='passthrough'
)



range_n_clusters = range(2, 10)
silhouette_scores = {}

print("Iniciando a busca pelo melhor número de clusters (k)...")

#Loop para encontrar o melhor K

for n_clusters in range_n_clusters:
    #Cria o pipeline final, adicionando o KMeans ao pré-processador
    #O pré-processador irá preparar os dados e o resultado será passado para o KMeans
    clusterer_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('clusterer', KMeans(n_clusters=n_clusters,
                               n_init=100,
                               random_state=42))
    ])

    #Executa o pipeline: pré-processa e clusteriza
    cluster_labels = clusterer_pipeline.fit_predict(df)

    #Para calcular a silhueta, precisamos dos dados JÁ TRANSFORMADOS
    #Podemos acessar o pré-processador treinado e transformar os dados
    X_transformed = clusterer_pipeline.named_steps['preprocessor'].transform(df)

    #Calcula o score de silhueta
    silhouette_avg = silhouette_score(X_transformed, cluster_labels)
    silhouette_scores[n_clusters] = silhouette_avg

    print(f"Para k = {n_clusters}, o Coeficiente de Silhueta é: {silhouette_avg:.4f}")

#Encontra o melhor K com base no maior score de silhueta
best_k = max(silhouette_scores, key=silhouette_scores.get)

print(f"\nMelhor k encontrado: {best_k} com um Score de Silhueta de {silhouette_scores[best_k]:.4f}")


Iniciando a busca pelo melhor número de clusters (k)...
Para k = 2, o Coeficiente de Silhueta é: 0.1758
Para k = 3, o Coeficiente de Silhueta é: 0.1957
Para k = 4, o Coeficiente de Silhueta é: 0.1542
Para k = 5, o Coeficiente de Silhueta é: 0.1623
Para k = 6, o Coeficiente de Silhueta é: 0.1456
Para k = 7, o Coeficiente de Silhueta é: 0.1466
Para k = 8, o Coeficiente de Silhueta é: 0.1450
Para k = 9, o Coeficiente de Silhueta é: 0.1540

Melhor k encontrado: 3 com um Score de Silhueta de 0.1957


Encontramos o melhor resultado em k = 3, porém devemos levar em consideração 3 fatores:
<br>
 1. silhueta é uma métrica dita "pessimista", onde se o valor dessa métrica é alto é um ótimo indicativo de que sua clusterização é adequada, quando silhueta não tem um resultado bom, é adequado aderir outras métricas também.
 2. Não fizemos praticamente nenhum pre processamento, logo muitos valores devem estar inadequados para uso.
 3. Os dados podem não ser esféricos ou elisoidais, levando a falha intriscica do kmeans, sendo necessario utilizar outro método de agrupamento como DBScan

Para resolver esses problemas devemos:
- Melhorar nosso pre processamento (moda normalmente é uma imputação ruim)
- Utilizar outras métricas também para definir o número de cluster.
- Testar **DBSCAN**, pois mesmo refinando o pre processamento e utilizando várias métricas pode continuar mostrando um agrupa inadequado e isso pode ser por conta de como os dados estão relacionados.

# Analise exploratória

Uma breve análise exploratória para encontrar informações sobre as variáveis

In [ ]:
print("\nInformações gerais do DataFrame combinado:")
df.info()

In [ ]:
df.describe()


In [ ]:
df.describe(include='object')


In [ ]:
df.head()

In [ ]:
print(df.isnull().sum())


Muitos NaN's principalmente em Work_Experience que deverão ser preenchidos

In [ ]:
df.drop(['Age', 'Work_Experience'], axis=1).nunique()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df_numeric = df.select_dtypes(include=['Int64', 'int64'])

sns.pairplot(df, hue='Spending_Score', diag_kind='kde')
plt.show()

Existe um aglomeramento de valores em Work_experience entre 0 e 2.5

Distribuição dos valores únicos nas colunas:

In [ ]:
import matplotlib.pyplot as plt

categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    counts = df[col].value_counts()
    percents = df[col].value_counts(normalize=True) * 100

    print(f"Valores únicos em {col}:")
    print(counts, "\n")

    plt.figure(figsize=(8,5))
    counts.plot(kind='bar')

    # Adiciona porcentagens acima das barras
    for i, (count, percent) in enumerate(zip(counts, percents)):
        plt.text(i, count + 0.5, f'{percent:.1f}%', ha='center', va='bottom')

    plt.title(f'Frequência dos valores únicos na coluna {col}')
    plt.xlabel(col)
    plt.ylabel('Contagem')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


Podemos perceber que em alguns atributos, existem valores que são raramente encontrados, principalmente em profession e em var_1. Isso pode afetar o agrupamento, então talvez seja necessario absorver algum valor em outro. Como Var_1 é categórica e não temos conhecimento suficiente de como funciona essa variável, não me arriscarei a agrupar ela. Profession por outro lado não é categórica, é nominal, portanto, tentarei unir marketing e homemaker em uma nova categoria "outros".

In [ ]:
categorias_para_agrupar = ["Marketing", "Homemaker"]

df["Profession_agrupada"] = df["Profession"].replace(categorias_para_agrupar, "Outros")

print(df["Profession_agrupada"].value_counts(normalize=True) * 100)


O que será feito:
- Normalização;
- Preenchimento de missing values;
- Remoção de outliers

Como Work_Experience e Family_Size não são inteiros, converteremos float para inteiro

In [ ]:
df['Work_Experience'] = df['Work_Experience'].astype('Int64')
df['Family_Size'] = df['Family_Size'].astype('Int64')

print(df.info())

Os unicos atributos que não possuem NaN's são: Age, Spending_Score, Gender
<br>

Atributos com Missing values:
- Ever_Married -> Object
- Graduated -> Object
- Profession -> Objetct
- Work_Experience -> Int64
- Family_Size -> Int64
- Var_1 -> Object

São 6 atributos onde temos 2 Int64 e 4 Object
para preenchimentos teremos que usar 2 tipos de modelos de imputação: Regression e classificação (ou regression com arredondamento).



# Pre processamento

## Preenchendo Nan's

### Técnica 1 - RandomForestClassifier



### Técnica 2 - Média

### Técnica 3 - KnnImputer

## Remoção de Outliers

### Técnica 1 - IQR

In [ ]:
##Remover outliers com base no iqr
# Principais atributos com outliers: Age, Credit amount, Duration
import pandas as pd

def remove_outliers_iqr(df, columns):
    """
    Remove registros com outliers com base no IQR para as colunas especificadas.
    """
    df_clean = df.copy()

    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.7 * IQR #1.8 pois temos muito pouco dados
        upper_bound = Q3 + 1.7 * IQR

        # Mantém apenas os registros dentro dos limites
        df_clean.loc[(df_clean[col] < lower_bound) | (df_clean[col] > upper_bound), col] = np.nan

    return df_clean


### Técnica 2 - Z score

## Técnica 3 - Manter os outliers

# Conclusão